In [1]:
import time, math, random
from copy import deepcopy
from IPython.display import clear_output
import numpy as np

MCTS is equivalent Monte Carlo control with UCB applied to the sub-MDP starting from the current state, where table lookup is represented by a tree. Since the action leads to deterministic results, $q(s,a)=v(s'_{\rm a})$. $\gamma=1$ and r=0 until endgame

# Barebone MCTS with UCT
https://youtu.be/ItMutbeOHtc?t=3922, https://www.youtube.com/watch?v=Bm7zah_LrmE

1. From each node (a node = a state of the board), repeat trying unexplored moves and rollout randomly unil gameover. The win/lose count is propagated back along the path (only above or at the level where new moves are being tried out -- for memory constraints) taken for each node
2. when all moves of the node have been tried at least once, *select* the one to try based on **Upper Confidence Bound** UCB (mean value + C\*uncertainty of the mean $\propto\sqrt{\frac{\log N_{\rm visit}}{n_{\rm child's visit}}}$). Repeat doing this until there is at least an untried move at the node
3. repeat 1&2 until desired time/iteration limit, then select the best move at root node (current real situation of the board) based on *mean value* of each move

4. consider some nodes may be reached from different ways; so keep the {move: child_node} dictionary. when selecting child node, return the corresponding move and child_node also (do not use parent of a node to backtrack)
5. ONLY for terminating (acyclic) games
6. Keep the nodes that (in principle) could be reached from the current state in the future. Game class need to implement `Reachable` wrt its key for this to work. Otherwise keep every node (memory...)
7. prune tree from the leaf. If searched to the end game already, we know the value of node exactly at that point, and can know if previous state will definitely (not) move towards there.

## Code: [mcts.ai](http://mcts.ai/code/python.html) recursive + node reuse + pruning

instance of game class is passed to `MCTS`'s `search` function, and must have:
* `playerToMove`: 1 or 2
* `GetMoves()`: returns list of (hashable) valid moves from current state
* `Move(m)`: execute the passed in move (which is guaranteed to be chosen from `GetMoves()`)
* `RollOut()`: randomly execuate the game until game over
* `GetResult(viewpoint)`: (after game over) returns the reward (1 or 0) from the specified viewpoint (1 or 2 -- corresponding to playerToMove)
* `key()`: returns a string fully and uniquely characterize the status of the game
* [Optional]`Reachable(k)`: returns whether (True/False) the game could in principle reach the state represented by the passed-in key `k`, which was generated by the `key()` function of another state. If not provided, prune tree by deleting nodes not visited during a search

In [2]:
class Node:
    """ Wins is always from the viewpoint of player Just Moved."""
    def __init__(self, state = None):
        self.childNodes = {} # {move:nextnode} the move that get *into* next node
        self.viewpoint = 3-state.playerToMove
        self.wins = 0  #for the player Just Moved
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        
    def UCTSelectChild(self,explore=1):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation. 
        """
        return max(list(self.childNodes.items()), 
                   key = lambda c: c[1].wins/c[1].visits + explore*math.sqrt(math.log(self.visits)/c[1].visits) )
    
    def AddChild(self, move, s, n):
        """ Remove m from untriedMoves and add a new child node for this move. if provided, use that  """
        self.untriedMoves.remove(move)
        self.childNodes[move] = n
        return n
    
    def Update(self, result, visits=None):
        """ update visit & win counts. (from the viewpoint of player Just Moved) """
        self.visits += 1
        self.wins += result # game results in the range [0.0, 1.0]

    def __repr__(self):
        return f"[W/V: {self.wins:6g}/{self.visits:6d} | UnXplrd: {len(self.untriedMoves)}] "

    def TreeToString(self, indent):
        s = "\n"+ "| "*indent + str(self)
        s += ''.join(c.TreeToString(indent+1)+str(m) for m,c in self.childNodes.items())
        return s

    def ChildrenToString(self):        
        return "\n".join(f'{str(m):8s}|{c}' for m,c in sorted(self.childNodes.items(),key=lambda e: e[1].wins/e[1].visits))
    
class MCTS():   #reuses nodes
    def __init__(self, game):
        self.nodes = {}               #store all nodes previously explored
        self.game = game
        self.inf = float('inf')

    def search(self, timemax=None, itermax=None, explore=1, verbose=0):
        key=self.game.key()
        if 'Reachable' in dir(self.game): #pruning by (game provided) finding which node will be reachable
            self.nodes = {k:v for k,v in self.nodes.items() if k==key or self.game.Reachable(k)}

        if self.rootnode.untriedMoves or len(self.rootnode.childNodes)>1:  #unimportant -- determine when to stop
            if itermax or timemax:
                start=time.time()
                i = 0
                while (timemax is None or time.time()<start+timemax) and\
                      (itermax is None or i<itermax):
                    i = i+100
                    for _ in range(100):
                        v=self.simulate(deepcopy(self.game),self.rootnode)
                    if abs(v)==self.inf: break
            else: #debug
                start=time.time()
                while True:
                    for _ in range(1000): v=self.simulate(deepcopy(self.game),self.rootnode)
                    print(time.time()-start,sum(c.visits for c in self.rootnode.childNodes.values()),
                          sorted(list(self.rootnode.childNodes.items()), 
                              key = lambda c: -c[1].wins/c[1].visits ),
                          end='\r')
                    if abs(v)==self.inf: break
                    
        moveToChild, bestChild = self.rootnode.UCTSelectChild(explore=0)
        if verbose==2:   print(self.rootnode.TreeToString(0))
        elif verbose==1: print(self.rootnode.ChildrenToString())
            
        return moveToChild    
    
    @property  #  called when accessing self.rootnode
    def rootnode(self):
        key = self.game.key()
        if key not in self.nodes:
            self.nodes[key] = Node(state = self.game)
        return self.nodes[key]
    
    def simulate(self,state,node,explore=1):
        if node.untriedMoves == [] and node.childNodes != {}: #fully expanded, non-terminal
            move,nnode = node.UCTSelectChild(explore) #descend (following UCB) 
            if nnode.wins==self.inf or nnode.wins==-self.inf: #if the best is -inf or inf already, can backprop
                node.Update(-nnode.wins)
                return -nnode.wins
            state.Move(move)
            v = self.simulate(state,nnode)
            if v==-self.inf: v=0 # if v==-inf, next mover will not choose this branch as it leads to loss.
                            # Note: other branch may have finite v as UCT  saw this branch as finite v before simulation
            node.Update(1-v)
            return 1-v
        elif node.untriedMoves != []: #not fully expanded: expand and then rollout
            move = random.choice(node.untriedMoves) 
            state.Move(move)
            k = state.key()
            if k in self.nodes:
                nnode = self.nodes[k]
            else:
                nnode = Node(state = state)
                self.nodes[k] = nnode
            node.AddChild(move,state,nnode)
            state.RollOut()
            v = state.GetResult(nnode.viewpoint) #not setting inf or -inf because there are indeterminism (unexplored moves)
            nnode.Update(v)
            node.Update(1-v)
            return 1-v
        elif node.childNodes == {}: #terminal
            v = state.GetResult(node.viewpoint)
            if v==1:    v=self.inf
            elif v==0:  v=-self.inf
            node.Update(v)
            return v
    
    def __simulate(self,state,node): #old, iterative version
        path = [node]
        #descend (following UCB) to the first branch not fully expanded (some moves not tried), or reaching end of game
        while node.untriedMoves == [] and node.childNodes != {}:
            move,node = node.UCTSelectChild(self.explore)
            if node.wins==float('inf') or node.wins==float('-inf'): #if the best is -inf or inf, can backprop already
                path[-1].Update(-node.wins)
                return
            state.Move(move)
            path += node,
        
        # if there are unexplored expand (add a childNode) and move the state into it 
        if node.untriedMoves != []: 
            move = random.choice(node.untriedMoves) 
            state.Move(move)
            k = state.key()
            if k in self.nodes:
                nextnode = self.nodes[k]
            else:
                nextnode = Node(state = state)
                
            node.AddChild(move,state,nextnode)
            self.nodes[k] = nextnode
            node = nextnode
            path += node,
        elif node.childNodes == {}: #nothing unexplored and is terminal: delete this child for its parent if loss
            reward = state.GetResult(node.viewpoint)
            if reward==1:
                node.Update(float('inf'))
            elif reward==0:
                node.Update(float('-inf'))

        # Rollout to END of a game randomly (not expanding childNodes -- just want to estimate the newly added node's value)
        state.RollOut()

        # state is now terminal; backpropagate this game's result to its path's nodes' win counts
        reward = state.GetResult(node.viewpoint)
        for n in path:
            if n.viewpoint == node.viewpoint:
                n.Update(reward)
            else:
                n.Update(1-reward)
        if self.prune: self.visitedNode|=set(path)

## [m,n,k-game](https://en.wikipedia.org/wiki/M,n,k-game)

In [3]:
#contains current state and who has last moved
import itertools
class OXO:
    def __init__(self,width=3,height=3,inarow=None,moves_per_round=None,end1=True): # game ends as one success
        self.playerToMove = 1
        self.width = width
        self.height = height
        self.board = [0]*width*height # 0 = empty, 1 = player 1, 2 = player 2
        self.empty = list(range(width*height))
        self.moves_per_round = moves_per_round or 1
        self.end1 = end1
        
        # winning linesrequires
        inarow = inarow or min(width,height)
        self.lines = [tuple(range(i*width+j,i*width+j+inarow)) for i in range(0,height) for j in range(width-inarow+1)]+\
                     [tuple(range(i*width+j,(i+inarow)*width+j,width)) for i in range(0,height-inarow+1) for j in range(width)]+\
                     [tuple(range(i*width+j,i*width+j+inarow*(width+1),width+1)) for i in range(height-inarow+1) for j in range(width-inarow+1)]+\
                     [tuple(range(i*width+j,i*width+j+inarow*(width-1),width-1)) for i in range(height-inarow+1) for j in range(inarow-1,width)]

    def Move(self, move):
        """ Update a state by carrying out the given move. Must also update playerToMove.  """
        for m in move:
            self.board[m] = self.playerToMove
            self.empty.remove(m)
        if self.end1 and self.GetResult(self.playerToMove) in [0,1]:  
            self.empty=[]
        self.playerToMove = 3 - self.playerToMove
        
    def GetMoves(self): # must clone as Node use this return as untriedMoves  # return self.empty[:]
        return list(itertools.combinations(self.empty,self.moves_per_round))
    
    def RollOut(self):
        possibleMoves = self.GetMoves()
        while possibleMoves:
            self.Move(random.choice(possibleMoves))
            possibleMoves = self.GetMoves()
    
    def GetResult(self, viewpoint):  # reward from `viewpoint`, in the range [0.0, 1.0]
        myscore = opposcore = 0
        for l in self.lines:
            one=self.board[l[-1]]
            if one!=0 and all(self.board[p]==one for p in l):
                if one == viewpoint:
                    myscore += 1
                else:
                    opposcore += 1
        if myscore>opposcore: return 1
        elif opposcore>myscore: return 0
        return 0.5

    def __repr__(self): # for output: 1 (X), 2 = (O)
        lines=[''.join("·XO"[self.board[i*self.width+j]] for j in range(self.width)) for i in range(self.height)]
        return '\n'.join(lines)    
    def key(self): #string containing all information for hashing the node
        return str(self.playerToMove)+''.join("·XO"[e] for e in self.board)
    def Reachable(self,key):  #for pruning Tree
        mykey = self.key()
        return all(ch in '·12' or ch==key[i] for i,ch in enumerate(mykey))

Fail to draw on 5,5,4, 2nd: after first player places at center, 2nd player should play at a position "diagonal" to the center (6,8,16,18). MCTS failed to identify these four as optimal until after ~10min

In [5]:
mcts = MCTS(OXO(width=5,height=5,inarow=4))
mcts.game.Move((12,)) # after this, found (6/8/16/18) is optimal after ~1,500,000 simulations
mcts.game.Move((11,)) # after this, found sure win policy after 260663 simulations
mcts.game.Move((16,)) # after this, found sure loss after 189564 simulations
# mcts.game.Move((8,))  # after this, found sure win policy after 38150 simulations
# mcts.game.Move((18,)) # after this, found sure loss after 25918 simulations
# mcts.game.Move((6,))  # after this, found sure win policy after 1601 simulations
mcts.search(timemax=None,verbose=0)

(7,)

In [ ]:
game = OXO(width=3,height=3,inarow=3,moves_per_round=1,end1=True) #traditional
# game = OXO(width=5,height=5,inarow=4) #http://boulter.com/ttt/
# game = OXO(width=8,height=8,inarow=4,moves_per_round=2,end1=False) #http://www.atksolutions.com/games/tictactoedeluxe.html
# game = OXO(width=15,height=15,inarow=5) #Gomoku
mcts = MCTS(game)
while mcts.game.GetMoves():
    if mcts.game.playerToMove==0:
#         clear_output()
        board=np.array([f'{("·XO"[s] if s else str(pos)):2s}' for pos,s in enumerate(mcts.game.board)]).reshape(mcts.game.height,mcts.game.width)
        print(board)
        n=mcts.rootnode
        print(n.ChildrenToString())
        print(f'choose from: {mcts.game.GetMoves()[:10]}...',end='')
        m = (int(input()),)   #2 moves (int(input()),int(input()))
    else:
        m = mcts.search(timemax=1,verbose=1)
    mcts.game.Move(m)
    print(mcts.game)
    print(len(mcts.nodes))
if mcts.game.GetResult(1) == 1:   print("Player 1(X) wins!")
elif mcts.game.GetResult(2) == 1: print("Player 2(O) wins!")
else: print("Nobody wins!")

## Connect4

In [110]:
class ConnectN:
    def __init__(self,width=7,height=6,inarow=4):
        self.playerToMove = 1
        self.width = width
        self.height = height
        self.board = [0]*width*height
        self.moves = list(range(width))
        self.firstEmpty = [height-1]*width
        
        # winning lines
        inarow = inarow or min(width,height)
        self.lines = [tuple(range(i*width+j,i*width+j+inarow)) for i in range(0,height) for j in range(width-inarow+1)]+\
                     [tuple(range(i*width+j,(i+inarow)*width+j,width)) for i in range(0,height-inarow+1) for j in range(width)]+\
                     [tuple(range(i*width+j,i*width+j+inarow*(width+1),width+1)) for i in range(height-inarow+1) for j in range(width-inarow+1)]+\
                     [tuple(range(i*width+j,i*width+j+inarow*(width-1),width-1)) for i in range(height-inarow+1) for j in range(inarow-1,width)]

    def Move(self, move):  #move=column idx
        assert move in self.moves
        self.board[self.firstEmpty[move]*self.width+move]=self.playerToMove
        self.firstEmpty[move] -= 1
        if self.firstEmpty[move]<0:
            self.moves.remove(move)
        if self.GetResult(self.playerToMove) in [0,1]:  #indicate game ended as someone won
            self.moves=[] 
        self.playerToMove = 3 - self.playerToMove
        
    def GetMoves(self):  # legal moves for next player. empty if game is over
        return self.moves[:]
    
    def RollOut(self):
        while self.moves:
            self.Move(random.choice(self.moves))
    
    def GetResult(self, viewpoint):  # reward from `viewpoint`, in the range [0.0, 1.0]
        for l in self.lines:
            one=self.board[l[-1]]
            if one!=0 and all(self.board[p]==one for p in l):
                if one == viewpoint:
                    return 1
                else:
                    return 0
        return 0.5

    def __repr__(self): # string representation: 1 (X), 2 = (O)
        s = ''
        for left in range(0,self.width*self.height,self.width):
            for sh in range(self.width):
                s+="·XO"[self.board[left+sh]]
            s+='\n'
        return s.strip()
    def key(self): #string containing all information for hashing the node
        return str(self.playerToMove)+''.join("·XO"[e] for e in self.board)
    def Reachable(self,key):  #for pruning Tree
        mykey = self.key()
        return all(ch in '12·' or ch==key[i] for i,ch in enumerate(mykey)) 

Fails to deduce that the 5th move (1st player) should be at center

In [ ]:
game = ConnectN(7,6,4) #https://connect4.gamesolver.org/
for _ in range(4): game.Move(3)
mcts = MCTS(game)
while mcts.game.GetMoves():
    if mcts.game.playerToMove==2:
#         clear_output()
        fn = lambda s: s if int(s)<7 else ''
        board=np.array([f'{("·XO"[s] if s else fn(str(pos))):2s}' for pos,s in enumerate(mcts.game.board)])
        print(board.reshape(mcts.game.height,mcts.game.width))
        print(mcts.rootnode.ChildrenToString())
        print('choose from:',end='')
        m = int(input(mcts.game.GetMoves()))
    else:
        m = mcts.search(timemax=10,verbose=1,explore=2)
    mcts.game.Move(m)
    print(mcts.game)
    print(len(mcts.nodes))
if mcts.game.GetResult(1) == 1:   print("Player 1(X) wins!")
elif mcts.game.GetResult(2) == 1: print("Player 2(O) wins!")
else: print("Nobody wins!")

## Othello

In [107]:
class Othello:
    def __init__(self, size = 8):  # size must be integral and even
        self.playerToMove = 1
        self.size = size
        self.board = [] # 0 = empty, 1 = player 1, 2 = player 2
        for y in range(self.size):
            self.board.append([0]*size)
        self.board[size//2][size//2] = self.board[size//2-1][size//2-1] = 2
        self.board[size//2][size//2-1] = self.board[size//2-1][size//2] = 1
        self.board = [e for l in self.board for e in l]

    def Move(self, move):
        if move is not None:
            (x,y)=divmod(move,self.size)
            assert self.IsOnBoard(x,y) and self.board[x*self.size+y] == 0
            m = self.GetAllSandwichedCounters(x,y)
            self.board[x*self.size+y] = self.playerToMove
            for (a,b) in m:
                self.board[a*self.size+b] = self.playerToMove
        self.playerToMove = 3 - self.playerToMove
    
    def GetMoves(self):
        emptypos = [pos for pos,e in enumerate(self.board) if e==0]
        if not emptypos:
            return []
        else:
            viable = [pos for pos in emptypos if self.ExistsSandwiched(*divmod(pos,self.size))] 
            if viable:
                return viable
            else:  #need to check if opponent also has no viable pos
                self.playerToMove = 3 - self.playerToMove
                oppoViable = [pos for pos in emptypos if self.ExistsSandwiched(*divmod(pos,self.size))]
                self.playerToMove = 3 - self.playerToMove
                if oppoViable:
                    return [None]
                else:
                    return []
    
    def RollOut(self):
        emptypos = [pos for pos,e in enumerate(self.board) if e==0]
        bad=0 
        while emptypos:
            random.shuffle(emptypos)
            for pos in emptypos:
                if self.ExistsSandwiched(*divmod(pos,self.size)):
                    self.Move(pos)
                    emptypos = [i for i,e in enumerate(self.board) if e==0]
                    break
            else:
                if bad<2: #when bad=2, both side cannot play, game ends
                    bad+=1
                    self.Move(None)
                else:
                    break

    def AdjacentEnemyDirections(self,x,y):# Speeds up GetMoves by only considering squares which are adjacent to an enemy-occupied square.
        return [(dx,dy) for (dx,dy) in [(0,+1),(+1,+1),(+1,0),(+1,-1),(0,-1),(-1,-1),(-1,0),(-1,+1)]
                        if self.IsOnBoard(x+dx,y+dy) and self.board[(x+dx)*self.size+y+dy] == 3-self.playerToMove]
    
    def ExistsSandwiched(self,x,y):# Is there at least one counter which would be flipped if my counter was placed at (x,y)? 
        for (dx,dy) in self.AdjacentEnemyDirections(x,y):
            x1=x+dx
            y1=y+dy
            while self.IsOnBoard(x1,y1) and self.board[x1*self.size+y1] == 3 - self.playerToMove:
                x1 += dx
                y1 += dy
            if self.IsOnBoard(x1,y1) and self.board[x1*self.size+y1] == self.playerToMove:
                return True
        return False
    
    def GetAllSandwichedCounters(self, x, y):# Is (x,y) a possible move (i.e. opponent counters are sandwiched between (x,y) and my counter in some direction)?
        sandwiched = []
        for (dx,dy) in self.AdjacentEnemyDirections(x,y):
            sandwiched.extend(self.SandwichedCounters(x,y,dx,dy))
        return sandwiched

    def SandwichedCounters(self, x, y, dx, dy):# Return the coordinates of all opponent counters sandwiched between (x,y) and my counter.
        x += dx
        y += dy
        sandwiched = []
        while self.IsOnBoard(x,y) and self.board[x*self.size+y] == 3 - self.playerToMove:
            sandwiched.append((x,y))
            x += dx
            y += dy
        if self.IsOnBoard(x,y) and self.board[x*self.size+y] == self.playerToMove:
            return sandwiched
        else:
            return [] # nothing sandwiched

    def IsOnBoard(self, x, y):
        return x >= 0 and x < self.size and y >= 0 and y < self.size
    
    def GetResult(self, viewpoint): #after gameover
        viewpointscore=oppositescore=0
        for e in self.board:
            if e==viewpoint: viewpointscore+=1
            elif e==3-viewpoint: oppositescore+=1
        if viewpointscore > oppositescore: return 1.0
        elif oppositescore > viewpointscore: return 0.0
        else: return 0.5 # draw

    def __repr__(self):
        s= ""
        for x in range(self.size):
            for y in range(self.size):
                s += "·XO"[self.board[x*self.size+y]]
            s += "\n"
        return s.strip()
    def key(self): #string containing all information for hashing the node
        return str(self.playerToMove)+''.join("·XO"[e] for e in self.board)
    def Reachable(self,key):  #for pruning Tree
        mykey = self.key()
        return key.count('·')<mykey.count('·') and \
               all(ch in '12·' or key[i]!='·' for i,ch in enumerate(mykey)) and\
               (mykey[1]=='·' or mykey[1]==key[1]) and\
               (mykey[self.size]=='·' or mykey[self.size]==key[self.size]) and\
               (mykey[-self.size]=='·' or mykey[-self.size]==key[-self.size]) and\
               (mykey[-1]=='·' or mykey[-1]==key[-1])

* timemax=60 beats Reversu Difficult
* timemax=15 beats Reversu Medium, lost to Reversu Difficult

In [ ]:
game = Othello(8)
mcts = MCTS(game)
while mcts.game.GetMoves():
    if mcts.game.playerToMove==2:
        clear_output()
        board=np.array([f'{("·XO"[e] if e else str(p)):2s}' for p,e in enumerate(mcts.game.board)]).reshape(8,8)
        print(board)
        print(mcts.rootnode.ChildrenToString())
        print('choose from:',end='')
        moves = mcts.game.GetMoves()
        if moves[0] is None: m = None
        else: m=int(input(moves))
    else:
        m = mcts.search(verbose=1,timemax=3+len(list(filter(None,mcts.game.board)))*0 )  #len...=number of pieces on board
    mcts.game.Move(m)
    print(mcts.game)
    print(len(mcts.nodes))
if mcts.game.GetResult(1) == 1:   print("Player 1(X) wins!")
elif mcts.game.GetResult(2) == 1: print("Player 2(O) wins!")
else: print("Nobody wins!")

# AlphaZero-style (guide tree search by policy+value network)

https://www.youtube.com/watch?v=ld28AU7DDB4
* The uncertainty term used in UCB is replaced by **Polynomial Upper Confidence Trees** PUCT $\propto P(s_i|s)\frac{\sqrt{N}}{1+n_i}$ (where N is the total visits of current state s, and n_i are visit counts of each of the next possible nodes). $P(s_i|s)$ is a probability output by policy network given current state
* Rollout replaced by the value estimation of the current state by the network. For a newly-expanded child node (c), use value network to compute all the values of its possible child nodes (c's c), used for computing the mean of the current (c) node, with weights given by the policy network.
* In AlphaGo, policy network is trained with human expert positions. Then policy network is then used to play against itself to generate positions + target (win/loss) for training the value network
* AlphaZero combines policy and value network into a single network. Repeat:
  1. training examples (state,policy of next move,win/loss) are generated by MCTS, used to train the policy+value network.
  2. Use the network trained to guide MCTS
  
Other tweaks
* in the PUCT, normalize (sum of all=1) and *then* exponentiate all counts before putting into formula $N^{1/\tau}$ and $n_i^{1/\tau}$, where $\tau=1$ for the first 10-100 moves (exploratory) and ~0 for later moves (only choose the best).
* After each iteration, pit against previous version of the net to make sure it has really improved

## [Code](http://web.stanford.edu/~surag/posts/alphazero.html) (modified)

`Game` class requires:
* `reset()` returns init board (numpy array passable to net)
* `actionSize` the largest possible number of moves (not given a state). e.g. Othello is size^2-4+1 (for pass)
* `getValidMoves()` returns np.array of 0/1 indicting the corresponding index action is a valid move or not at the moment
* `board1`: Board in such a way that the one next move is always same set of numbers in the numpy array. e.g., for othello if playerToMove=2, return board*(-1). e.g. in chess if playerToMove=2, exchange pieces number 1-16 with number 17-32
* `symConfig()`: input a policy (prob dist), generate tuples of (board**1**,policy) due to board symmetry

### MCTS given guidance from Net, generates (states,policies,values) through self-plays

In [114]:
class Node:
    def __init__(self, state = None):
        self.childNodes = {} # {move:nextnode} the move that get *into* next node
        self.policy = {}     # {move:prob}
        self.viewpoint = 3-state.playerToMove
        self.wins = 0        #for the player Just Moved = self.viewpoint
        self.visits = 0
        
    def UCTSelectChild(self,explore=1):
        mv = max(self.policy, 
                 key = lambda k: self.childNodes[k].wins/(self.childNodes[k].visits) + \
                       explore*self.policy[k]*math.sqrt(self.visits)/(1+self.childNodes[k].visits)\
                       if k in self.childNodes else explore*self.policy[k]*math.sqrt(self.visits+1e-8)
                )
        return mv
    
    def AddChild(self, move, s, n):
        self.childNodes[move] = n
        return n
    
    def Update(self, result):
        self.visits += 1
        self.wins += result # game results in the range [0.0, 1.0]
    def __repr__(self):        return f"[W/V: {self.wins:6g}/{self.visits:6d}] "
    def ChildrenToString(self):return "\n".join(f'{str(m):8s}|{c}' for m,c in sorted(self.childNodes.items(),key=lambda e: e[1].wins/e[1].visits))

In [138]:
from collections import deque
from itertools import count
import numpy as np

class MCTS():
    def __init__(self, game, net):
        self.game = game
        self.net = net
        self.numMCTSSims=25
        self.inf = float('inf')
        self.nodes = {}
    def search(self, timemax=None, itermax=None, explore=1, verbose=False):
        key=self.game.key()
        if 'Reachable' in dir(self.game):
            self.nodes = {k:v for k,v in self.nodes.items() if k==key or self.game.Reachable(k)}

        if len(self.rootnode.policy)>1:  #unimportant -- determine when to stop
            if itermax or timemax:
                start=time.time()
                i = 0
                while (timemax is None or time.time()<start+timemax) and\
                      (itermax is None or i<itermax):
                    i = i+100
                    for _ in range(100):
                        v=self.simulate(deepcopy(self.game),self.rootnode)
                    if v==self.inf: break
            else: #debug
                start=time.time()
                while True:
                    for _ in range(1000): self.simulate(deepcopy(self.game),self.rootnode)
                    print(time.time()-start,sum(c.visits for c in self.rootnode.childNodes.values()),
                          sorted(list(self.rootnode.childNodes.items()), 
                              key = lambda c: -c[1].wins/c[1].visits ),
                          end='\r')
                    
        moveToChild = self.rootnode.UCTSelectChild(explore=0)
        if verbose: print(self.rootnode.ChildrenToString())

        return moveToChild    
    
    def net_predict(self,board1,valids): # returns a dict of {act:prob} for the valid moves, and a value of broad1's position
        probs, v = self.net.predict(board1[np.newaxis])
        probs = probs[0]*valids
        sum_probs = np.sum(probs)
        if sum_probs > 0:
            probs /= sum_probs 
        else:# print("All valid moves were masked, do workaround.")
            probs = probs + valids
            probs /= np.sum(probs)
        return {a:probs[a] for a,val in enumerate(valids) if val}, v[0]
    
    def selfplay(self,temperature=lambda s: 1 if s<5 else 0): #train batch from one complete game
        """
        This function executes one episode of self-play, starting with player 1.
        As the game is played, each turn is added to trainExamples. After the game
        ends, the outcome of the game is used to assign values to each example.
        It uses a temp=1 if episodeStep < tempThreshold, and thereafter
        uses temp=0.
        Returns:
            trainExamples: a list of examples of the form (canonicalBoard,pi,v)
                           pi is the MCTS informed policy vector, v is +1 if
                           the player eventually won the game, else -1.
        """
        trainExamples = []
        board = self.game.reset()

        for movestep in count():
            tau = temperature(movestep)

            probs = self.getActionProb(tau)
            
            for board1,policy in self.game.symConfig(probs):
                trainExamples += (self.game.playerToMove, board1.astype(np.float32), policy),  #value target not known yet

            action = np.random.choice(len(probs), p=probs)
            self.game.Move(action)

            if not any(self.game.getValidMoves()):
                r = self.game.GetResult(self.game.playerToMove)
                return [(e[1],e[2],r if e[0]==self.game.playerToMove else 1-r) for e in trainExamples]
        
    @property  #  called when accessing self.rootnode
    def rootnode(self):
        key = self.game.key()
        if key not in self.nodes:
            self.nodes[key] = Node(state = self.game)
            probs, v = self.net_predict(self.game.board1,self.game.getValidMoves())
            self.nodes[key].policy = probs
        return self.nodes[key]

    def getActionProb(self, temp=1): #Returns: probs for next move by performing multiple searches starting from current state#
        for _ in range(self.numMCTSSims):
            self.simulate(deepcopy(self.game), self.rootnode)
            
        get_count=lambda c: 1e8 if c.wins==self.inf else 1e-8 if c.wins==-self.inf else c.visits #1e-8 to avoid all-zero
        counts = [get_count(self.rootnode.childNodes[a]) if a in self.rootnode.childNodes else 0 for a in range(self.game.actionSize)]
        if temp==0:
            bestA = np.argmax(counts)
            probs = [0]*len(counts)
            probs[bestA]=1
        else:
            counts = [x**(1./temp) for x in counts]
            probs = [x/float(sum(counts)) for x in counts]
        return probs

    def simulate(self, state, node, explore=1): #not aiming to fully expand every node
        """
        At a newly expanded state, the net is called to return an initial P and v for the state.
        If the leaf node is a terminal state, the value is v. the value is propogated up the search path
        """
        if node.policy == {}: #terminal
            v = state.GetResult(node.viewpoint)
            if v==1:    v=self.inf
            elif v==0:  v=-self.inf
            node.Update(v)
            return v
        else:                 # non-terminal
            move = node.UCTSelectChild(explore) #descend (following UCB)
            state.Move(move)
            if move in node.childNodes:
                nnode = node.childNodes[move]
                if nnode.wins==self.inf or nnode.wins==-self.inf: #if the best is -inf or inf already, can backprop
                    node.Update(-nnode.wins)
                    return -nnode.wins
            else:
                k = state.key()                
                if k in self.nodes:
                    nnode = self.nodes[k]
                else:
                    nnode = Node(state = state)
                    valid = state.getValidMoves()
                    if np.any(valid):
                        probs, v = self.net_predict(state.board1,valid)
                        nnode.policy = probs
                    self.nodes[k] = nnode
                node.AddChild(move,state,nnode)
            
            v = self.simulate(state,nnode)
            if v==-self.inf: v=0 # if v==-inf, next mover will not choose this branch as it leads to loss.
                                 # Note: other branch may have finite v as UCT  saw this branch as finite v before simulation
#             nnode.Update(v)
            node.Update(1-v)
            return 1-v

### Train net with dataset (states,policies,values) given by the self-plays of MCTS

In [8]:
import tensorflow as tf
@tf.function
def train_step(net, boardstates, policies_target, values_target, optimizer):
    with tf.GradientTape() as tape:
        policies, values = net(boardstates, training=True)
        loss_p = tf.losses.softmax_cross_entropy(policies_target, policies)
        loss_v = tf.losses.mean_squared_error(values_target, values)
        combined_loss = loss_p + loss_v

    grad = tape.gradient(combined_loss, net.trainable_variables)
    optimizer.apply_gradients(zip(grad, net.trainable_variables))

In [ ]:
# Example training script with a **Game** class and a **net**
from collections import deque
train_set = deque(maxlen=200000)
mcts = MCTS(Game(),net)
optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

for _ in range(100):
    train_set += mcts.selfplay()
    for s,p,v in tf.data.Dataset.from_tensor_slices(zip(*train_set)).shuffle(1000).batch(64):
        train_step(net, s,p,v, optimizer)

### MCTS using trained Net: given state, generate next move

In [ ]:
# Example running) through script with a **Game** class and a **net**
game = OXO(5,5,4)
mcts = MCTS(game,net)
while np.any(mcts.game.getValidMoves()):
    m = mcts.search(timemax=10,verbose=1)
    mcts.game.Move(m)
    print(mcts.game, len(mcts.nodes))
if mcts.game.GetResult(1) == 1:   print("Player 1(X) wins!")
elif mcts.game.GetResult(2) == 1: print("Player 2(O) wins!")
else: print("Nobody wins!")

## [m,n,k-game](https://en.wikipedia.org/wiki/M,n,k-game)

In [139]:
class OXO:
    def __init__(self,width=3,height=3,inarow=None,end1=True): # game ends as one success
        self.playerToMove = 1
        self.width = width
        self.height = height
        self.end1 = end1
        self.actionSize = width*height
        self.reset()        
        inarow = inarow or min(width,height)
        self.lines = [tuple(range(i*width+j,i*width+j+inarow)) for i in range(0,height) for j in range(width-inarow+1)]+\
                     [tuple(range(i*width+j,(i+inarow)*width+j,width)) for i in range(0,height-inarow+1) for j in range(width)]+\
                     [tuple(range(i*width+j,i*width+j+inarow*(width+1),width+1)) for i in range(height-inarow+1) for j in range(width-inarow+1)]+\
                     [tuple(range(i*width+j,i*width+j+inarow*(width-1),width-1)) for i in range(height-inarow+1) for j in range(inarow-1,width)]
    
    @property
    def board1(self):
        return self.board.reshape(self.height,self.width) if self.playerToMove==1\
               else np.mod(3-self.board,3).reshape(self.height,self.width)
        
    def reset(self):
        self.playerToMove = 1
        self.board = np.zeros(self.width*self.height,dtype=np.int32)
        self.empty = list(range(self.width*self.height))
        
    def getValidMoves(self): 
        return np.clip(1-self.board,0,None) if self.empty else np.zeros(self.actionSize)

    def symConfig(self,pi):
        board1 = self.board1
        pi=np.array(pi).reshape(self.height,self.width)
        return [(board1,pi.flatten()),
                (np.rot90(board1,1),np.rot90(pi,1).flatten()),
                (np.rot90(board1,2),np.rot90(pi,2).flatten()),
                (np.rot90(board1,3),np.rot90(pi,3).flatten()),
                (np.flip(board1,0), np.flip(pi,0).flatten()),
                (np.rot90(np.flip(board1,0),1), np.rot90(np.flip(pi,0),1).flatten()),
                (np.rot90(np.flip(board1,0),2), np.rot90(np.flip(pi,0),2).flatten()),
                (np.rot90(np.flip(board1,0),3), np.rot90(np.flip(pi,0),3).flatten()),
               ]

    def Move(self, move):
        assert move in self.empty
        self.board[move] = self.playerToMove
        self.empty.remove(move)
        if self.end1 and self.GetResult(self.playerToMove) in [0,1]:  
            self.empty=[]
        self.playerToMove = 3 - self.playerToMove

    def GetResult(self, viewpoint):
        myscore = opposcore = 0
        for l in self.lines:
            one=self.board[l[-1]]
            if one!=0 and all(self.board[p]==one for p in l):
                if one == viewpoint: myscore += 1
                else: opposcore += 1
        if myscore>opposcore: return 1
        elif opposcore>myscore: return 0
        return 0.5

    def __repr__(self): return '\n'.join(''.join("·XO"[self.board[i*self.width+j]] for j in range(self.width)) for i in range(self.height))    
    def key(self): return str(self.playerToMove)+''.join("·XO"[e] for e in self.board)
    def Reachable(self,key): return all(ch in '·12' or ch==key[i] for i,ch in enumerate(self.key()))

In [140]:
import tensorflow as tf
from tensorflow.keras.layers import *

def genNet(oxo):
    inp = tf.keras.layers.Input(shape=[oxo.height,oxo.width])
    x = Reshape([oxo.height,oxo.width,1])(inp)
    x = Activation('relu')(BatchNormalization(axis=3)(Conv2D(512, 2, padding='same')(x)))         # batch_size  x board_x x board_y x num_channels
    x = Activation('relu')(BatchNormalization(axis=3)(Conv2D(512, 2, padding='same')(x)))         # batch_size  x board_x x board_y x num_channels
    x = Activation('relu')(BatchNormalization(axis=3)(Conv2D(512, 2, padding='valid')(x)))        # batch_size  x (board_x-2) x (board_y-2) x num_channels
    x = Activation('relu')(BatchNormalization(axis=3)(Conv2D(512, 2, padding='valid')(x)))        # batch_size  x (board_x-4) x (board_y-4) x num_channels
    x = Flatten()(x)       
    
    s_fc1 = Dropout(.3)(Activation('relu')(BatchNormalization(axis=1)(Dense(1024)(x))))  # batch_size x 1024
    s_fc2 = Dropout(.3)(Activation('relu')(BatchNormalization(axis=1)(Dense(512)(x))))   # batch_size x 1024
    p = Dense(oxo.actionSize, activation='softmax', name='pi')(s_fc2)
    v = Dense(1, activation='tanh', name='v')(s_fc2)
    
    m = tf.keras.Model(inputs=inp, outputs=[p,v])
    m.compile(loss=['categorical_crossentropy','mean_squared_error'], optimizer=tf.keras.optimizers.Adam(0.001))
    return m

In [ ]:
import tensorflow as tf
from collections import deque

game = OXO(5,5,4)
# net = genNet(game)
train_set = deque(maxlen=12800)
mcts = MCTS(game,net)

while True:
    train_set += mcts.selfplay()
    clear_output()
    s,p,v=map(np.asarray,zip(*random.sample(train_set,len(train_set))))
    net.fit(x = s, y = [p, v], verbose=1, batch_size=64)

Train on 12800 samples
12800/12800 [==============================] - 44s 3ms/sample - loss: 0.5285 - pi_loss: 0.4986 - v_loss: 0.0299


single game

In [ ]:
game = OXO(5,5,4)
mcts = MCTS(game,net)
for s,p,v in mcts.selfplay():
    print(s,'\n',p.reshape(5,5),v)

full game playout

In [ ]:
game = OXO(5,5,4)
mcts = MCTS(game,net)
while np.any(mcts.game.getValidMoves()):
    m = mcts.search(timemax=10,verbose=1)
    mcts.game.Move(m)
    print(mcts.game, len(mcts.nodes))
if mcts.game.GetResult(1) == 1:   print("Player 1(X) wins!")
elif mcts.game.GetResult(2) == 1: print("Player 2(O) wins!")
else: print("Nobody wins!")

## [Connect4](https://github.com/plkmo/AlphaZero_Connect4)

## [Othello](https://github.com/suragnair/alpha-zero-general)